In [1]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [2]:
l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX", "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT","L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()
    
l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

In [3]:
o_columnnames = ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERSTATUS", "O_TOTALPRICE", "O_ORDERDATE", "O_ORDERPRIORITY", "O_CLERK", "O_SHIPPRIORITY", "O_COMMENT"]

for i in range(len(o_columnnames)):
    o_columnnames[i] = o_columnnames[i].lower()
    
o_data_types = {
    'o_orderkey': int,
    'o_custkey': int,
    'o_orderstatus': str,
    'o_totalprice': float,
    'o_orderpriority': str,
    'o_clerk': str,
    'o_shippriority': int,
    'o_comment': str
}

o_parse_dates = ['o_orderdate']

In [4]:
c_columnnames = ["C_CUSTKEY", "C_NAME", "C_ADDRESS", "C_NATIONKEY", "C_PHONE", "C_ACCTBAL", "C_MKTSEGMENT", "C_COMMENT"]

for i in range(len(c_columnnames)):
    c_columnnames[i] = c_columnnames[i].lower()
    
c_data_types = {
    'c_custkey': int,
    'c_name': str,
    'c_address': str,
    'c_nationkey': int,
    'c_phone': str,
    'c_acctbal': float,
    'c_mktsegment': str,
    'c_comment': str
}

c_parse_dates = []

In [5]:
# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../tpch-pgsql-master/data/load/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

In [6]:
# Don't set indexes, as we can't access them with Pandas selection!
orders = pd.read_table("../tpch-pgsql-master/data/load/orders.tbl.csv", sep="|", names=o_columnnames, dtype=o_data_types, parse_dates=o_parse_dates)

In [7]:
# Don't set indexes, as we can't access them with Pandas selection!
customer = pd.read_table("../tpch-pgsql-master/data/load/customer.tbl.csv", sep="|", names=c_columnnames, dtype=c_data_types, parse_dates=c_parse_dates)

In [8]:
start_t = time.time()

df_filter_1 = customer[['c_custkey', 'c_name', 'c_address', 'c_nationkey', 'c_phone', 'c_acctbal', 'c_mktsegment', 'c_comment']]
df_filter_2 = orders[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey']]
df_filter_3 = lineitem[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
# Here we added the start of the agg
df_group_1 = df_filter_3 \
    .groupby(['l_orderkey']) \
    .agg(sum_quantity=("l_quantity", "sum")
)
# And the filtering afterwards
df_group_1 = df_group_1[df_group_1.sum_quantity > 300]
df_merge_1 = df_filter_2.merge(df_group_1, left_on="o_orderkey", right_on="l_orderkey")
df_merge_1 = df_merge_1[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey']]
df_sort_1 = df_merge_1.sort_values(by=['o_custkey'], ascending=[True])
df_sort_1 = df_sort_1[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey']]
df_merge_2 = df_filter_1.merge(df_sort_1, left_on="c_custkey", right_on="o_custkey")
df_merge_2 = df_merge_2[['c_name', 'c_custkey', 'o_orderkey', 'o_orderdate', 'o_totalprice']]
# Here we added in l_orderkey
df_filter_4 = lineitem[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_merge_3 = df_merge_2.merge(df_filter_4, left_on="o_orderkey", right_on="l_orderkey")
df_merge_3 = df_merge_3[['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice', 'l_quantity']]
df_sort_2 = df_merge_3.sort_values(by=['c_custkey', 'o_orderkey'], ascending=[True, True])
df_sort_2 = df_sort_2[['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice', 'l_quantity']]
# Add in aggregation
df_group_2 = df_sort_2 \
    .groupby(['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice']) \
    .agg(suml_quantity=("l_quantity", "sum")
)
#Change and prop forward the column name
df_group_2 = df_group_2[['suml_quantity']]
df_sort_3 = df_group_2.sort_values(by=['o_totalprice', 'o_orderdate'], ascending=[False, True])
df_sort_3 = df_sort_3[['suml_quantity']]
df_limit_1 = df_sort_3[['suml_quantity']]
result = df_limit_1.head(100)

print(len(result))
print(result)

print("---" * 5)
print(time.time() - start_t)
print("---" * 5)

57
                                                                  suml_quantity
c_custkey o_orderkey c_name             o_orderdate o_totalprice               
128120    4722021    Customer#000128120 1994-04-07  544089.09             323.0
144617    3043270    Customer#000144617 1997-02-12  530604.44             317.0
13940     2232932    Customer#000013940 1997-04-13  522720.61             304.0
66790     2199712    Customer#000066790 1996-09-30  515531.82             327.0
46435     4745607    Customer#000046435 1997-07-03  508047.99             309.0
15272     3883783    Customer#000015272 1993-07-28  500241.33             302.0
146608    3342468    Customer#000146608 1994-06-12  499794.58             303.0
96103     5984582    Customer#000096103 1992-03-16  494398.79             312.0
24341     1474818    Customer#000024341 1992-11-15  491348.26             302.0
137446    5489475    Customer#000137446 1997-05-23  487763.25             311.0
107590    4267751    Customer#0001075

In [10]:
start_t = time.time()

df_filter_1 = customer[['c_custkey', 'c_name', 'c_address', 'c_nationkey', 'c_phone', 'c_acctbal', 'c_mktsegment', 'c_comment']]
df_filter_2 = orders[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey']]
df_filter_3 = lineitem[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_group_1 = df_filter_3 \
    .groupby(['l_orderkey']) \
    .agg(
        suml_quantity=("l_quantity", "sum"),
    )
df_group_1 = df_group_1[df_group_1.suml_quantity > 300]
df_merge_1 = df_filter_2.merge(df_group_1, left_on="o_orderkey", right_on="l_orderkey")
df_merge_1 = df_merge_1[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey']]
df_sort_1 = df_merge_1.sort_values(by=['o_custkey'], ascending=[True])
df_sort_1 = df_sort_1[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey']]
df_merge_2 = df_filter_1.merge(df_sort_1, left_on="c_custkey", right_on="o_custkey")
df_merge_2 = df_merge_2[['c_name', 'c_custkey', 'o_orderkey', 'o_orderdate', 'o_totalprice']]
df_filter_4 = lineitem[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_merge_3 = df_merge_2.merge(df_filter_4, left_on="o_orderkey", right_on="l_orderkey")
df_merge_3 = df_merge_3[['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice', 'l_quantity']]
df_sort_2 = df_merge_3.sort_values(by=['c_custkey', 'o_orderkey'], ascending=[True, True])
df_sort_2 = df_sort_2[['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice', 'l_quantity']]
df_group_2 = df_sort_2 \
    .groupby(['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice']) \
    .agg(
        suml_quantity=("l_quantity", "sum"),
    )
df_group_2 = df_group_2[['suml_quantity']]
df_sort_3 = df_group_2.sort_values(by=['o_totalprice', 'o_orderdate'], ascending=[False, True])
df_sort_3 = df_sort_3[['suml_quantity']]
df_limit_1 = df_sort_3[['suml_quantity']]
result = df_limit_1.head(100)

print(len(result))
print(result)

print("---" * 5)
print(time.time() - start_t)
print("---" * 5)

57
                                                                  suml_quantity
c_custkey o_orderkey c_name             o_orderdate o_totalprice               
128120    4722021    Customer#000128120 1994-04-07  544089.09             323.0
144617    3043270    Customer#000144617 1997-02-12  530604.44             317.0
13940     2232932    Customer#000013940 1997-04-13  522720.61             304.0
66790     2199712    Customer#000066790 1996-09-30  515531.82             327.0
46435     4745607    Customer#000046435 1997-07-03  508047.99             309.0
15272     3883783    Customer#000015272 1993-07-28  500241.33             302.0
146608    3342468    Customer#000146608 1994-06-12  499794.58             303.0
96103     5984582    Customer#000096103 1992-03-16  494398.79             312.0
24341     1474818    Customer#000024341 1992-11-15  491348.26             302.0
137446    5489475    Customer#000137446 1997-05-23  487763.25             311.0
107590    4267751    Customer#0001075

In [ ]:
li_aggr = lineitem \
    .groupby(["l_orderkey"]) \
    .agg(sum_quantity=("l_quantity", "sum"))

li_filt = li_aggr[li_aggr.sum_quantity > 300].reset_index()
li_proj = li_filt[["l_orderkey"]]

ord_filt = orders[orders.o_orderkey.isin(li_proj["l_orderkey"])]

cu_proj = customer[["c_custkey", "c_name"]]
ord_cu_join = pd.merge(cu_proj, ord_filt, left_on = "c_custkey", right_on = "o_custkey", how = "inner")
ord_cu_join = ord_cu_join[["o_orderkey", "c_name", "c_custkey", "o_orderdate", "o_totalprice"]]

li_ord_join = pd.merge(ord_cu_join, lineitem, left_on = "o_orderkey", right_on = "l_orderkey", how = "inner")

result = li_ord_join \
    .groupby(["c_name", "c_custkey", "o_orderkey", "o_orderdate", "o_totalprice"]) \
    .agg(sum_quantity=("l_quantity", "sum"))
    
result = result.sort_values(by=['o_totalprice', 'o_orderdate'], ascending=[False, True])
result = result.head(100)

print(len(result))
print(result)

57
                                                                  sum_quantity
c_name             c_custkey o_orderkey o_orderdate o_totalprice              
Customer#000128120 128120    4722021    1994-04-07  544089.09            323.0
Customer#000144617 144617    3043270    1997-02-12  530604.44            317.0
Customer#000013940 13940     2232932    1997-04-13  522720.61            304.0
Customer#000066790 66790     2199712    1996-09-30  515531.82            327.0
Customer#000046435 46435     4745607    1997-07-03  508047.99            309.0
Customer#000015272 15272     3883783    1993-07-28  500241.33            302.0
Customer#000146608 146608    3342468    1994-06-12  499794.58            303.0
Customer#000096103 96103     5984582    1992-03-16  494398.79            312.0
Customer#000024341 24341     1474818    1992-11-15  491348.26            302.0
Customer#000137446 137446    5489475    1997-05-23  487763.25            311.0
Customer#000107590 107590    4267751    1994-11-0